## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-31-22-52-10 +0000,nyt,"Twenty Years Later, Atrocities Haunt Darfur Again",https://www.nytimes.com/2025/10/31/world/afric...
1,2025-10-31-22-45-03 +0000,startribune,Judges order Trump administration to use emerg...,https://www.startribune.com/judges-could-rule-...
2,2025-10-31-22-41-50 +0000,nyt,JD Vance Says He Hopes Usha Vance Embraces Chr...,https://www.nytimes.com/2025/10/31/us/politics...
3,2025-10-31-22-40-14 +0000,nyt,Ohio Republicans Push Through New Districts in...,https://www.nytimes.com/2025/10/30/us/politics...
4,2025-10-31-22-35-07 +0000,nyt,Top Senators Say Pentagon Has Not Shared Key I...,https://www.nytimes.com/2025/10/31/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2316/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,50
29,new,13
14,shutdown,13
17,he,12
216,government,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
316,2025-10-30-23-41-00 +0000,wsj,A day after President Trump vowed to resume te...,https://www.wsj.com/politics/national-security...,119
142,2025-10-31-16-22-00 +0000,wsj,President Trump urged Senate Republicans to en...,https://www.wsj.com/politics/policy/filibuster...,113
173,2025-10-31-14-56-14 +0000,nypost,Trump’s call to go ‘nuclear’ to end government...,https://nypost.com/2025/10/31/us-news/trumps-c...,112
271,2025-10-31-04-57-01 +0000,nypost,Trump demands Senate Republicans invoke ‘nucle...,https://nypost.com/2025/10/31/us-news/trump-de...,108
295,2025-10-31-00-58-02 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...,106


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
316,119,2025-10-30-23-41-00 +0000,wsj,A day after President Trump vowed to resume te...,https://www.wsj.com/politics/national-security...
64,81,2025-10-31-20-26-00 +0000,wsj,The federal government pays out $100 billion a...,https://www.wsj.com/politics/policy/what-to-kn...
208,51,2025-10-31-11-41-06 +0000,nypost,Prince Andrew thrown ‘under the bus’ by royal ...,https://nypost.com/2025/10/31/world-news/princ...
295,49,2025-10-31-00-58-02 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...
142,48,2025-10-31-16-22-00 +0000,wsj,President Trump urged Senate Republicans to en...,https://www.wsj.com/politics/policy/filibuster...
315,46,2025-10-30-23-42-12 +0000,nypost,Amazon shares soar as AI demand boosts cloud r...,https://nypost.com/2025/10/30/business/amazon-...
37,41,2025-10-31-21-28-57 +0000,nyt,Trump Administration Must Pay SNAP Benefits Du...,https://www.nytimes.com/2025/10/31/us/politics...
111,38,2025-10-31-18-12-27 +0000,bbc,'No inch here was spared': Hurricane Melissa d...,https://www.bbc.com/news/videos/ce3xxzg09gro?a...
190,34,2025-10-31-13-36-12 +0000,nypost,Hochul attempts to do damage control after cat...,https://nypost.com/2025/10/31/us-news/zohran-m...
265,32,2025-10-31-08-00-00 +0000,cbc,"Whether the target is drugs or Maduro, U.S. co...",https://www.cbc.ca/news/world/us-hegseth-three...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
